##### 불균형 클래스
##### 클래스 불균형에 대한 이해
##### 클래스 가중치 사용
##### 리셈플링 기법
##### 적절한 평가지표

In [25]:
# 불균형 데이터 생성(1:9)  악성 0 : 양성 1
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn.pipeline import Pipeline
np.random.seed(42)

data = load_breast_cancer()
X, y = data.data,  data.target
# 악성을 소수 클래스로 생성

print(f'악성 양성의 오리지널 비율 : {np.unique(y,return_counts=True)}')
m_index =  np.where(y == 0)[0]  # index 반환  악성
b_index =  np.where(y == 1)[0]

# 악성은 일부만, 양성은 더 많이사용
# 악성의 30%만 사용, 양성은 전체  1.5:8.5
size_30 = int(len(m_index) * 0.2)
selected_m_index = np.random.choice(m_index,size=size_30,replace=False)
selected_b_index = b_index

concatenate_selected_index = np.concatenate([selected_m_index,  selected_b_index])
np.random.shuffle(concatenate_selected_index)

X_imb = X[concatenate_selected_index]
y_imb = y[concatenate_selected_index]

# 클래스 분포 확인
unique, counts = np.unique(y_imb,return_counts=True)
print('클래스 분포 :')
for label, count in zip(unique,counts):
    percentage = count / len(y_imb) * 100
    print(f'클래스 {label} : {count}개 ({percentage:.1f}%)')

# 불균형인 상태로 진행
# 스케일링 정규화 StandardSacaler
# LogisticRegression
# pipe 
# 평가는 class report

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

x_train,x_test,y_train,y_test = train_test_split(X_imb, y_imb,stratify=y_imb, test_size=0.2,random_state=42)
print('1. 기본 모델 (불균형 무시)')
pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', LogisticRegression(random_state=42,max_iter=1000))
])
pipe.fit(x_train, y_train)
print( classification_report(y_test, pipe.predict(x_test),target_names=['악성(0)','양성(1)'] )  )

악성 양성의 오리지널 비율 : (array([0, 1]), array([212, 357]))
클래스 분포 :
클래스 0 : 42개 (10.5%)
클래스 1 : 357개 (89.5%)
1. 기본 모델 (불균형 무시)
              precision    recall  f1-score   support

       악성(0)       1.00      0.88      0.93         8
       양성(1)       0.99      1.00      0.99        72

    accuracy                           0.99        80
   macro avg       0.99      0.94      0.96        80
weighted avg       0.99      0.99      0.99        80



In [28]:
print('불균형 해결 : 클래스 가중치 사용')
# 기존파이프라인의 clf 이름의 객체의 파라메터를 조정
from copy import deepcopy
pipe_weight = deepcopy(pipe)
pipe_weight.set_params(clf__class_weight = 'balanced')

# pipe_weight = Pipeline([
#     ('scaler',StandardScaler()),
#     ('clf', LogisticRegression(random_state=10,max_iter=1000,class_weight = 'balanced'))
# ])


pipe_weight.fit(x_train,y_train)
print( classification_report(y_test, pipe_weight.predict(x_test)  )   )

print("가중치 계산")
n_samples = len(y_train)
n_classes = 2
class_counts = np.bincount(y_train)
for i in range(n_classes):
    weight = n_samples / (n_classes * class_counts[i])
    print(f' 클래스 :{i} : {weight:.3f}')    

불균형 해결 : 클래스 가중치 사용
              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.99      1.00      0.99        72

    accuracy                           0.99        80
   macro avg       0.99      0.94      0.96        80
weighted avg       0.99      0.99      0.99        80

가중치 계산
 클래스 :0 : 4.691
 클래스 :1 : 0.560


In [40]:
from sklearn.ensemble import RandomForestClassifier
print(' 불균형 : RandomFores (균형모드)')
pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(class_weight='balanced',random_state=42))
])
pipe_rf.fit(x_train, y_train)
y_pred = pipe_rf.predict(x_test)
print( classification_report(y_test, y_pred) )

 불균형 : RandomFores (균형모드)
              precision    recall  f1-score   support

           0       1.00      0.62      0.77         8
           1       0.96      1.00      0.98        72

    accuracy                           0.96        80
   macro avg       0.98      0.81      0.87        80
weighted avg       0.96      0.96      0.96        80



In [ ]:
# 오버 or 언더 셈플링 - 직접 실행..  SMOTE  SMOTEEN